In [1]:
import pandas as pd
import numpy as np
import os 
import re
import cx_Oracle
import string
import warnings
import sqlite3
from sqlite3 import Error

warnings.simplefilter(action='ignore', category=FutureWarning) #suppress warning from numpy operator for string comparison
#fpath = 'H:/2019_Projects/QC_absRead/absorbance'
fpath = 'M:/2019/NPSG projects/PPSPE_QC/absorbance'

def corrABS(df1,df2,fx,frmExclFile=False):
    if frmExclFile:
        df1_cr = df1.iloc[0::2,:] #1st cell for crude
        df2_fx7 = df2.iloc[1::2,:] #last cell for 6/7th fx
    else: #it will contain all columns, so deal with data wrangling differently
        if fx:
            if int(fx) == 7:
                index_col = (0,2)
            elif int(fx) == 6:
                index_col = (1,2)
            df1_cr = df1.iloc[0::2,0::2]
            df2_fx = df2.iloc[1::2,index_col[0]::index_col[1]]
    dft = df1_cr.transpose().iloc[::-1].transpose().iloc[::-1] #flip
    dflstt = [item for sublist in dft.values.tolist() for item in sublist]
    dflst1 = [item for sublist in df1_cr.values.tolist() for item in sublist]
    dflst2 = [item for sublist in df2_fx.values.tolist() for item in sublist]
    dfarr_cr = convertFloatArray(dflst1)
    dfarr_fx = convertFloatArray(dflst2)
    dfarr_t = convertFloatArray(dflstt)
    return (np.corrcoef(dfarr_cr,dfarr_fx)[0][1],np.corrcoef(dfarr_t,dfarr_fx)[0][1])

def checkVarType(var):
    if isinstance(var,list):
        var = np.array(var)
    return var

def convertFloatArray(var):
    a = checkVarType(var)
    if np.isin(a,'OVRFLW').any():
        a[a=='OVRFLW']=0
        a=a.astype(np.float)
        a[a==0]=np.amax(a)        
        return a
    else:
        return a.astype(np.float)

def splStr(strg):
    t_strg = strg.split('.')
    return t_strg[0]

def readata(filename,fx=None,filterData=False,ninesixwell=False):
    fileNameString = splStr(filename)
    collectionNum = fileNameString[:8]
    suffix = fileNameString[8:]
    if filterData:
        if int(fx)==7 or suffix == '100':
            df = pd.read_excel(fpath+'/'+filename,usecols=list(range(3,25,2)))
        elif int(fx)==6:
            df = pd.read_excel(fpath+'/'+filename,usecols=list(range(4,25,2)))
    elif ninesixwell:
        df = pd.read_excel(fpath+'/'+'96DW/'+filename)       
    else:
        df = pd.read_excel(fpath+'/'+filename,usecols=list(range(3,15)))       
    return (df,collectionNum,suffix)

def dfRowColDict(df):
    df.index = [i for i in string.ascii_uppercase[:len(df)]]
    colSuffix = [f'0{i}' if len(str(i)) <2 else str(i) for i in df.columns]
    rowDict = {k:v for k,v in zip(range(len(df)),string.ascii_uppercase[:len(df)])}
    colDict = {k:v for k,v in zip(range(len(df.columns)),colSuffix)}
    return (rowDict,colDict)

def genPropArray(df,pltcode,pltsuf,wavelh,dictDat):
    dfarray = convertFloatArray(df.values)
    dim = dfarray.shape
    wellIDarray = np.zeros(dim).astype(str)
    mdim = dim[0]*dim[1]
    multiply = lambda x:[x]*mdim
    properties = [np.array(i).reshape(dim) for i in list(map(multiply,[pltcode,pltsuf,wavelh]))]
    for i in range(dim[0]):
        for j in range(dim[1]):
            wellIDarray[i][j] = '{}{}'.format(dictDat[0][i],dictDat[1][j])
    properties.append(wellIDarray)
    properties.append(dfarray)
    return properties

def create_connection(db_file):
    try:
        conn = sqlite3.connect('file:{}?mode=rwc'.format(db_file),uri=True) #Read, Write & Create mode, so connecting to a non-existing database will cause it to be created
        print('sqlite3 version: '+sqlite3.version+'\n==============\n')
        return conn
    except Exception as e:
        print(e)

def create_table(conn,tableName):
    exec_1 = 'CREATE TABLE IF NOT EXISTS '
    exec_2 = ''' (PLATE_CODE INTEGER,PLATE_SUFFIX TEXT,PLATE_POSITION TEXT, READING REAL,WAVELENGTH TEXT)'''
    c = conn.cursor()
    c.execute(exec_1 + tableName + exec_2)
    
def data_entry(conn,listOfVals,tblname):
    exec_1 = '''INSERT INTO ''' 
    exec_2 = ''' (PLATE_CODE,PLATE_SUFFIX,PLATE_POSITION,READING,WAVELENGTH) VALUES (:1,:2,:3,:4,:5)'''
    c = conn.cursor()
    c.execute(exec_1+tblname+exec_2,listOfVals)
    #print(exec_1+tblname+exec_2,listOfVals)
    
def enterDatDB(File,wavelh,conn,tblname,ninesixwell=False):
    try:
        dataPltCodeSuffix=readata(File,ninesixwell) #read filename and generate dataframe
        dictDat=dfRowColDict(dataPltCodeSuffix[0]) #generate dictionary values for well ID
        allData = genPropArray(dataPltCodeSuffix[0],str(dataPltCodeSuffix[1]),dataPltCodeSuffix[2],wavelh,dictDat)
        for i in range(allData[0].shape[0]):
            for j in range(allData[0].shape[1]):
                values = [allData[0][i][j], #pltcode
                          allData[1][i][j], #pltsuffix
                          allData[3][i][j], #plt position
                          allData[4][i][j], #reading
                          allData[2][i][j].item()] #wavelength
                with conn:
                    data_entry(conn,values,tblname)
    except cx_Oracle.IntegrityError as e:
        print(f'{values[0]} : {e}')
        pass
                
def dataStatEntr(con,plateCode):
    try:
        c = con.cursor()
        q100_1 = """SELECT PLATE_SUFFIX,READING FROM WELL_ABSORBANCE WHERE (PLATE_CODE =  """
        q100_2 = """ AND PLATE_SUFFIX = '100') ORDER BY PLATE_POSITION"""
        q100 = c.execute(q100_1+plateCode+q100_2)
        r100 = q100.fetchall()
        q200_1 = """SELECT PLATE_SUFFIX,READING FROM WELL_ABSORBANCE WHERE (PLATE_CODE = """ 
        q200_2 = """ AND PLATE_SUFFIX = '200') ORDER BY PLATE_POSITION"""
        q200 = c.execute(q200_1+plateCode+q200_2)
        r200 = q200.fetchall()
        rone = [i[1] for i in r100]
        rtwo = [i[1] for i in r200]
        colRone = int(len(rone)/16)
        colRtwo = int(len(rtwo)/16)
        df1 = pd.DataFrame(np.array(rone).reshape((16,colRone)))
        df2 = pd.DataFrame(np.array(rtwo).reshape((16,colRtwo)))
        rcorr7 = corrABS(df1,df2,fx=7)
        rcorr6 = corrABS(df1,df2,fx=6)
        qcorrNorm7 = c.execute("""INSERT INTO ABSORBANCE_STATS (PLATE_CODE,STATISTIC,FRACTION,STAT_TYPE) VALUES (:1,:2,:3,:4)""",[plateCode,rcorr7[0],7,'Normal Correlation'])
        qcorrNorm6 = c.execute("""INSERT INTO ABSORBANCE_STATS (PLATE_CODE,STATISTIC,FRACTION,STAT_TYPE) VALUES (:1,:2,:3,:4)""",[plateCode,rcorr6[0],6,'Normal Correlation'])
        qcorrRot7 = c.execute("""INSERT INTO ABSORBANCE_STATS (PLATE_CODE,STATISTIC,FRACTION,STAT_TYPE) VALUES (:1,:2,:3,:4)""",[plateCode,rcorr7[1],7,'Rotated Correlation'])
        qcorrRot6 = c.execute("""INSERT INTO ABSORBANCE_STATS (PLATE_CODE,STATISTIC,FRACTION,STAT_TYPE) VALUES (:1,:2,:3,:4)""",[plateCode,rcorr6[1],6,'Rotated Correlation'])
        #print('{}:{},{}'.format(plateCode,rcorr7,rcorr6))
    except cx_Oracle.IntegrityError as e:
        print(f'{plateCode} : {e}')
        pass

In [19]:
#test data
df=readata('14170909100.xlsx',1)[0]
dictDat=dfRowColDict(df)
prop = genPropArray(df,'14170909','100',660,dictDat)
with open('heatmpdat.csv','w') as f:
    for i in range(prop[4].shape[0]):
        for j in range(prop[4].shape[1]):
            #print(prop[4][i,j])
            f.write(str(prop[4][i,j])+'\n,')

In [18]:
for fi in npltcode:
    try:
        enterDatDB(fi,660,con,'WELL_ABSORBANCE')
    except Exception as e:
        print(f'{fi} : {e}')

In [ ]:
#CHECK DF SHAPE
for i in nlst:
    try:
        df = pd.read_excel(fpath +'/96DW/'+i)
        num = df.shape
        print(f'{i} : {num}')
    except Exception as e:
        print(f'{i}, ** {e}')

In [20]:
for i in pltcodeNew:
    try:
        dataStatEntr(con,i)
    except ValueError as e:
        print(f'{i}:{e}')
con.commit()

In [61]:
lstfiles = [fi for fi in os.listdir(fpath) if re.match('\w',fi)] #not include ~.files (temp files?)

host = 'DTPIV1.NCIFCRF.GOV'
port = 1523
servname = 'PROD.NCIFCRF.GOV'
user = 'NPSG'
passwd = 'P2wC9Gq3r4'

dsn_tns = cx_Oracle.makedsn(host,port,service_name=servname)
con = cx_Oracle.connect(user,passwd, dsn_tns)

#c = con.cursor()
#query = c.execute("SELECT * FROM WELL_ABSORBANCE")
#r = query.fetchall()
#colnames = list(map(lambda x: x[0], c.description))
#for row in r:
#    print(row)    

In [26]:
import time
lstfilesDates=[]
for i in lstfiles:
    #print(os.path.getctime(fpath+'/'+i))
    timestamp = [i for i in time.ctime(os.path.getctime(fpath+'/'+i)).split(' ') if i]
    lstfilesDates.append(timestamp)
alst=[]
for i,j in zip(lstfiles,lstfilesDates):
    time = j[1]+' '+j[2]
    if time == 'Feb 7':
        alst.append(splStr(i)[:8])

In [16]:
c = con.cursor()
query=c.execute("SELECT DISTINCT PLATE_CODE FROM WELL_ABSORBANCE")
r = query.fetchall()
pltcodeComplt = [pcode[0] for pcode in r]    
pltcodeNew = set([i for i in [splStr(j)[0:8] for j in lstfiles if j.endswith('.xlsx') and not j.startswith('mw')] if i not in pltcodeComplt])

In [ ]:
#CHECK IF EACH PLATE CODE HAS TWO FILES (100,200)
addsuff = lambda x,y: (f'{x}100.xlsx',f'{y}200.xlsx')
for i in pltcodeNew:
    iname = addsuff(i,i)
    if all([os.path.isfile(fpath+'/'+iname[0]),os.path.isfile(fpath+'/'+iname[1])]):
        try:
            enterDatDB(iname[0],660,con,'WELL_ABSORBANCE')
            enterDatDB(iname[1],660,con,'WELL_ABSORBANCE')
            dataStatEntr(con,i)
        except Exception as e:
            print(f'{i} : {e}')
    else:
        print(f'{i} : False')
con.commit()

In [16]:
#addsuff = lambda x,y: (f'{x}100.xlsx',f'{y}200.xlsx')
tuplefi = [addsuff(i,i) for i in pltcodeNew]
npltcode = list(sum(tuplefi, ()))

In [ ]:
c = conn.cursor()
c.execute("SELECT * FROM wellAbs")
colnames = list(map(lambda x: x[0], c.description))
result_set = c.fetchall()
for row in result_set:
    print(row)

In [15]:
#re-format 96W to 384 for correlation:
c = con.cursor()
c.execute('''SELECT M.*,ROW_96 || LPAD(COL_96, 2, '0') FROM WELL_MAP M''')
r = c.fetchall()
dfmapSQL = pd.DataFrame(r)
c.execute("SELECT COLUMN_NAME FROM USER_TAB_COLUMNS WHERE TABLE_NAME = 'WELL_MAP'")
colnames = c.fetchall()
suffix = colnames.pop(0)[0]
dfmapSQL.columns = [i[0] for i in colnames]+[suffix,'96_PAD0']

In [4]:
fxdict = {'90':'0','91':'1', '92':'2', '93':'3', '94':'4', '95':'5', '96':'6', '97':'7'}

def map96wTo384_entrDat(file,dfmapSQL):
    dfRaw = readata(file,ninesixwell=True)
    df = dfRaw[0].iloc[:,1:-1]
    fx_suffix = splStr(file)[9:]
    if int(fx_suffix) <94:
        pltsuffix = '100'
    else:
        pltsuffix = '200'
    fx_suffix384 = fxdict[fx_suffix]
    rwcolDict = dfRowColDict(df)
    dfArr = genPropArray(df,dfRaw[1],dfRaw[2],660,rwcolDict)
    dfArr[1] = np.repeat(pltsuffix,dfArr[1].size).reshape((dfArr[1].shape[0],dfArr[1].shape[1]))
    for i in range(dfArr[3].shape[0]):
        for j in range(dfArr[3].shape[1]):
            dfq = dfmapSQL[dfmapSQL['96_PAD0'].str.match(dfArr[3][i,j])]
            dfq = dfq[(dfq['SUFFIX']==pltsuffix) & (dfq['FRACTION']==fx_suffix384)]
            if int(dfq['COL_384'].values[0]) < 10:
                zeropad = '0'
            else:
                zeropad = ''
            wellID = f"{dfq['ROW_384'].values[0]}{zeropad}{dfq['COL_384'].values[0]}"
            dfArr[3][i,j] = wellID
    return dfArr

def enterDat96(File,con,tblname):
    try:
        allData = map96wTo384_entrDat(File,dfmapSQL)
        for i in range(allData[0].shape[0]):
            for j in range(allData[0].shape[1]):
                values = [allData[0][i][j], #pltcode
                          allData[1][i][j], #pltsuffix
                          allData[3][i][j], #plt position
                          allData[4][i][j], #reading
                          allData[2][i][j].item()] #wavelength
                with con:
                    data_entry(con,values,tblname)
    except cx_Oracle.IntegrityError as e:
        print(f'{values[0]} : {e}')
        pass

In [57]:
pltcode=['14170902','14171105']
lst96files = [fi for fi in os.listdir(fpath+'/96DW') if re.match(pltcode[0],fi)] + [fi for fi in os.listdir(fpath+'/96DW') if re.match(pltcode[1],fi)]
lst96files

['14170902290.xlsx',
 '14170902291.xlsx',
 '14170902292.xlsx',
 '14170902293.xlsx',
 '14170902295.xlsx',
 '14170902296.xlsx',
 '14170902297.xlsx',
 '14170902394.xlsx',
 '14171105290.xlsx',
 '14171105291.xlsx',
 '14171105292.xlsx',
 '14171105293.xlsx',
 '14171105294.xlsx',
 '14171105295.xlsx',
 '14171105296.xlsx',
 '14171105297.xlsx']

In [ ]:
for i in lst96files:
    try:
        with con:
            #enterDat96(i,con,'WELL_ABSORBANCE')
            dataStatEntr(con,splStr(i)[:8])
            con.commit()
    except Exception as e:
        print(e)

In [45]:
conn = create_connection('wellAbs.db')

sqlite3 version: 2.6.0



In [46]:
nlst = [fi for fi in os.listdir(fpath+'/96DW') if re.match('14171105',fi)] 

In [53]:
c = conn.cursor()
r = c.execute("SELECT READING FROM WELLABS WHERE PLATE_CODE = '14171105' AND PLATE_SUFFIX = '100'")
res100 = r.fetchall()
r = c.execute("SELECT READING FROM WELLABS WHERE PLATE_CODE = '14171105' AND PLATE_SUFFIX = '200'")
res200 = r.fetchall()

In [57]:
r100 = [i[0] for i in res100]
r200 = [i[0] for i in res200]
df1 = pd.DataFrame(np.array(r100).reshape((16,int(len(r100)/16))))
df2 = pd.DataFrame(np.array(r200).reshape((16,int(len(r200)/16))))

In [63]:
corrABS(df1,df2,6)

(0.05772054161285061, 0.14728111120014437)

In [64]:
corrABS(df1,df2,7)

(0.03431238042146955, 0.3520139995211177)

In [62]:
lst = ['14170110','14170208','14170403','14161007','14170404','14170300','14170902','14170501']
c = con.cursor()
for i in lst:
    r = c.execute(f"SELECT * FROM WELL_ABSORBANCE WHERE PLATE_CODE='{i}'")
    res = r.fetchall()
    lres = len(res)
    if lres > 0:
        print(f'{i} : {lres} -- OKAY\n')
    else:
        print(f'{i} : {lres} -- PROBLEM CHILD\n')
    

14170110 : 384 -- OKAY

14170208 : 704 -- OKAY

14170403 : 704 -- OKAY

14161007 : 384 -- OKAY

14170404 : 704 -- OKAY

14170300 : 704 -- OKAY

14170902 : 704 -- OKAY

14170501 : 704 -- OKAY



In [ ]:
r = c.execute("SELECT DISTINCT PLATE_CODE FROM WELL_ABSORBANCE")
pltcodelst = [i[0] for i in r.fetchall()]
for i in pltcodelst:
    r = c.execute(f"SELECT COUNT(*) FROM WELL_ABSORBANCE WHERE PLATE_CODE = '{i}'")
    count = r.fetchall()[0][0]
    print(f"{i} : {count}")

In [43]:
nlst=set([splStr(fi)[:8] for fi in os.listdir(fpath+'/96DW') if re.match('\w',fi)] )

In [56]:
for i in nlst:
    r = c.execute(f"SELECT COUNT(*) FROM WELL_ABSORBANCE WHERE PLATE_CODE = '{i}'")
    count = r.fetchall()[0][0]
    lstOfdf=[]
    for j in [fi for fi in os.listdir(fpath+'/96DW') if re.match(i,fi)]:
        df = readata(j,ninesixwell=True)
        df = df[0].iloc[:,1:-1]
        lstOfdf.append(df.shape)
    dfDict = {i:lstOfdf}
    print(f"{i} : {count} : {lstOfdf}")

14170504 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170515 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170403 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170511 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170516 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170404 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170501 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170902 : 384 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14171105 : 384 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170208 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170510 : 704 : [(8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11), (8, 11)]
14170300 :